# Chapter 1: Initial Query with ChatGPT

Dear user, you are part of a consulting firm tasked to: \
[1] identify opportunities for design improvements for a specific product \
[2] conduct benchmarking with competitors whenever appropriate.

In this initial stage, we will initialize the **product** \
and gather initial insights and directions via querying a LLM (**ChatGPT**).

### TO DO SECTION

In [33]:
'''
Dear user, enter your Product here!
'''

product = "Scoot 787"

In [34]:
""" Use this for a fresh set up (Comment if not needed) """
import os
import shutil

if os.path.exists('support'):                     # Delete 'support' folder if it exists
    shutil.rmtree('support')

### RUN AS INTENDED (DO NOT CHANGE ANYTHING.)

#### Set Up

In [35]:
! pip install python-dotenv

DEPRECATION: google-images-search 1.4.6 has a non-standard dependency specifier click>=7.0<=8.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of google-images-search or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [36]:
""" Set up OpenAI API key """
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")

In [37]:
search_terms = product

In [38]:
""" Create directory """
import os

try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

try:                                              # Create directory to store ChatGPT
    os.makedirs("support/%s/%s" % (search_terms, "ChatGPT"))
    print("Directory ChatGPT created")
except FileExistsError:
    print("Directory ChatGPT exists")

Directory Scoot 787 created
Directory ChatGPT created


#### DESIGN OPPORTUNITIES QUERY with ChatGPT

In [39]:
""" Design Opportunities Query with ChatGPT """
import pickle
import pandas as pd
import json

from openai import OpenAI
client = OpenAI()

question =  f"Identify and rank ten opportunities for design improvements with components in {product}."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"), "wb"))

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)

try:
    components = list(data.keys())
    descriptions = list(data.values())

    df = pd.DataFrame({'Component': components, 'Description': descriptions})

    print("Design Opportunities Query with ChatGPT:")
    print(df)
except:
    print("Invalid Response. Please regenerate.")


Design Opportunities Query with ChatGPT:
                        Component                             Description
0                           Seats             Improve comfort and legroom
1  In-flight Entertainment System                Upgrade for more options
2                  Cabin Lighting             Enhance for better ambience
3                      Lavatories         Increase number and cleanliness
4                Overhead Storage               Maximize space efficiency
5                          Galley             Optimize for faster service
6                Air Conditioning  Improve for better temperature control
7                   Window Shades        Upgrade for better light control
8                     Aisle Space               Widen for easier movement
9                 Emergency Exits    Improve visibility and accessibility


## BENCHMARKING QUERY with ChatGPT

In [40]:
""" BENCHMARKING QUERY with ChatGPT """

""" Get 3 competitors """
import pickle
import pandas as pd
import json
import ast

from openai import OpenAI
client = OpenAI()

question =  f"Identify three competitors of the {product}."
template = " Express the answer only as a Python list with the product name (do not add numbering). If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"), "wb"))

""" Benchmarking """
print("Benchmarking Query with ChatGPT:")
competitors_string = pd.read_pickle("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"))
competitors = ast.literal_eval(competitors_string)

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)
components = list(data.keys())

client = OpenAI()

for i in range(len(competitors)):
    question = f"Using the {product}, conduct design benchmarking with {competitors[i]} with regards to these ten physical components: {components}."
    template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised comparison explanation with a maximum of 40 characters. If you don't know the answer to the question, strictly state 'I don't know.'."
    prompt = question + template

    chat_completion = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,  # adjust persona and temperature
    )

    result = chat_completion.choices[0].message.content

    pickle.dump(result, open("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]), "wb"))

    data_string = pd.read_pickle("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]))
    benchmarking_data = json.loads(data_string)
    try:
        components = list(benchmarking_data.keys())
        descriptions = list(benchmarking_data.values())

        df = pd.DataFrame({'Component': components, 'Benchmarking': descriptions})

        print(competitors[i])
        print(df)
        print("\n")
    except:
        print("Invalid Response. Please regenerate.")

Benchmarking Query with ChatGPT:
Airbus A330
                        Component                     Benchmarking
0                           Seats     Comparable comfort and space
1  In-flight Entertainment System          Scoot 787 more advanced
2                  Cabin Lighting            787 has mood lighting
3                      Lavatories  Comparable in size and features
4                Overhead Storage              787 has larger bins
5                          Galley    787 has more efficient layout
6                Air Conditioning   787 uses more efficient system
7                   Window Shades       787 has electronic dimming
8                     Aisle Space           Comparable aisle space
9                 Emergency Exits       Comparable number and size


Boeing 777
                        Component                    Benchmarking
0                           Seats  787 has more comfortable seats
1  In-flight Entertainment System              Comparable systems
2      